In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from dataclasses import dataclass, field
from typing import Dict, List
import warnings
warnings.filterwarnings('ignore')

import findspark
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Row
import pyspark
from OOS_helperfun import *

In [3]:
findspark.init()

In [4]:
%%time
"""
Build the SparkSession
"""
# getOrCreate(): get the current Spark session or to create one if there is none running
spark = SparkSession.builder \
   .master("local") \
   .appName("OOS Model") \
   .config("spark.executor.memory", "1gb") \
   .getOrCreate()
   
sc = spark.sparkContext

CPU times: user 22.9 ms, sys: 16.1 ms, total: 39 ms
Wall time: 7.46 s


#### Load and clean data

In [5]:
%%time
## load and clean data
df = spark.read.csv("../data/rowData/Row_data.csv", header=True)
df, date_generated = clean_data(df, spark)

Data processing finished.
CPU times: user 83.4 ms, sys: 23.3 ms, total: 107 ms
Wall time: 32.4 s


In [6]:
# save cleaned dataout, but we can jump this step. 
%%time
df.toPandas().to_csv('../data/cleanedData/cleaned_dataset.csv', 
                             index=False, encoding='utf-8')

CPU times: user 1.39 s, sys: 190 ms, total: 1.58 s
Wall time: 1min


## OOS check

In [7]:
%%time
output_data = check_OOS_by_rules(df, date_generated, spark)

CPU times: user 43.5 s, sys: 553 ms, total: 44 s
Wall time: 1min 7s


In [8]:
## Output dataset
output_data.to_csv('../data/output/OOS_SKU.csv', index=False, encoding='utf-8')

In [ ]:
## Analysis OOS 7 days product and output the result
OOS_7_days_item_analysis = output_OOS_7_days_analysis(df, output_data, category)
file_name = '../../data/output/OOS_7_days_analysis/OOS_7_days_'+category+'.csv'
OOS_7_days_item_analysis.to_csv(file_name, index=False, encoding='utf-8')   